In [2]:
import pandas as pd
receive_path = "/home/bentoaz/race_simulation/data/race_stats_sca copy.feather"
df_race = pd.read_feather(receive_path)
df_race

/home/bentoaz/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


FileNotFoundError: [Errno 2] No such file or directory: '/home/bentoaz/race_simulation/data/race_stats_sca copy.feather'

In [5]:
import pandas as pd

source_path = "/home/bentoaz/race_simulation/data/cost_stats_vec.feather"
df_cost = pd.read_feather(source_path)

# df_cost = df_cost.drop(["Theta_a1",	"Theta_b1",	"Theta_c1",	"Theta_a2",	"Theta_b2",	"Theta_c2", "Action_space1", "Action_space2", "Scenario", "P2 Vector Cost"],axis=1)
df_cost

,Theta_a1,Theta_b1,Theta_c1,Theta_a2,Theta_b2,Theta_c2,action1,Action_space1,action2,Action_space2,...,State1_v,State1_phi,State1_b,State2_x,State2_y,State2_v,State2_phi,State2_b,Scenario,P2 Vector Cost
0,1,1,1,0.001,0.0,0.0,7,9,7,9,...,5.15,3.141593,0.000000,816.766783,728.090054,5.15,2.773093,0.000000,close_tail,True
1,1,1,1,0.001,0.0,0.0,8,9,8,9,...,12.65,3.141593,0.008727,796.185367,736.037333,12.65,2.773093,0.008727,close_tail,True
2,1,1,1,0.001,0.0,0.0,4,9,7,9,...,15.00,3.460310,0.000000,756.281294,743.517690,20.15,3.129271,0.000000,close_tail,True
3,1,1,1,0.001,0.0,0.0,4,9,6,9,...,15.00,3.460310,0.000000,701.015489,744.198668,22.50,3.129271,-0.008727,close_tail,True
4,1,1,1,0.001,0.0,0.0,5,9,5,9,...,15.00,3.460310,0.008727,647.222886,758.600396,22.50,2.638366,0.008727,close_tail,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1,1,1,1.000,1.0,1.0,5,9,4,9,...,15.00,5.914835,0.008727,743.556897,85.754443,22.50,6.485236,0.000000,inside_edge,True
14996,1,1,1,1.000,1.0,1.0,3,9,5,9,...,15.00,6.242105,-0.008727,798.662614,97.042596,22.50,6.485236,0.008727,inside_edge,True
14997,1,1,1,1.000,1.0,1.0,5,9,4,9,...,15.00,5.914835,0.008727,848.773868,121.330787,22.50,6.976141,0.000000,inside_edge,True
14998,1,1,1,1.000,1.0,1.0,5,9,4,9,...,15.00,6.242105,0.008727,892.050454,157.264053,22.50,6.976141,0.000000,inside_edge,True


In [49]:
import pandas as pd
import os

def stack_n_rows(input_path, output_path, n=30):
    """
    Transform a dataset by stacking every n rows into a single row,
    and append new stacked rows column-wise to existing rows if file exists.

    Parameters:
        input_path (str): Path to input CSV/Feather/Parquet file.
        output_path (str): Path to save transformed file (Feather in this example).
        n (int): Number of rows to stack into one.
    """
    # Load new dataset
    df = pd.read_feather(input_path)
    df = df.drop(["Theta_a1","Theta_b1",	"Theta_c1",	"Theta_a2",	"Theta_b2",	"Theta_c2", "Action_space1", "Action_space2", "Scenario", "P2 Vector Cost"],axis=1)

    # Ensure row count is divisible by n (drop leftovers)
    num_full_chunks = len(df) // n
    df = df.iloc[:num_full_chunks * n]

    # Collect transformed rows
    stacked_rows = []
    for i in range(0, len(df), n):
        chunk = df.iloc[i:i+n].reset_index(drop=True)

        # Flatten the chunk into a single row
        flat_row = {}
        for row_idx in range(n):
            for col in df.columns:
                flat_row[f"{col}_{row_idx+1}"] = chunk.at[row_idx, col]
        stacked_rows.append(flat_row)
        print(f"Processed chunk {i // n + 1}")

    # New stacked DataFrame
    new_df = pd.DataFrame(stacked_rows).reset_index(drop=True)

    # If output file exists -> merge column-wise
    if os.path.exists(output_path):
        old_df = pd.read_feather(output_path).reset_index(drop=True)

        # Ensure same number of rows for concat
        min_rows = min(len(old_df), len(new_df))
        old_df = old_df.iloc[:min_rows].reset_index(drop=True)
        new_df = new_df.iloc[:min_rows].reset_index(drop=True)

        combined_df = pd.concat([old_df, new_df], axis=1)
    else:
        combined_df = new_df

    # Save combined DataFrame
    combined_df.to_feather(output_path)
    print(f"Column-wise merged & saved to {output_path}")

# Example usage
stack_n_rows(source_path, receive_path, n=30)

Processed chunk 1
Processed chunk 2
Processed chunk 3
Processed chunk 4
Processed chunk 5
Processed chunk 6
Processed chunk 7
Processed chunk 8
Processed chunk 9
Processed chunk 10
Processed chunk 11
Processed chunk 12
Processed chunk 13
Processed chunk 14
Processed chunk 15
Processed chunk 16
Processed chunk 17
Processed chunk 18
Processed chunk 19
Processed chunk 20
Processed chunk 21
Processed chunk 22
Processed chunk 23
Processed chunk 24
Processed chunk 25
Processed chunk 26
Processed chunk 27
Processed chunk 28
Processed chunk 29
Processed chunk 30
Processed chunk 31
Processed chunk 32
Processed chunk 33
Processed chunk 34
Processed chunk 35
Processed chunk 36
Processed chunk 37
Processed chunk 38
Processed chunk 39
Processed chunk 40
Processed chunk 41
Processed chunk 42
Processed chunk 43
Processed chunk 44
Processed chunk 45
Processed chunk 46
Processed chunk 47
Processed chunk 48
Processed chunk 49
Processed chunk 50
Processed chunk 51
Processed chunk 52
Processed chunk 53
Pr

In [50]:
df_race = pd.read_feather(receive_path)
df_race.columns.to_list()

['Race Number',
 'Passes P1',
 'Passes P2',
 'Collisions',
 'Choices',
 'Proportion Ahead P1',
 'Proportion Ahead P2',
 'Win P1',
 'Win P2',
 'Progress P1',
 'Progress P2',
 'Out of Bounds P1',
 'Out of Bounds P2',
 'Progress Cost P1',
 'Progress Cost P2',
 'Bounds Cost P1',
 'Bounds Cost P2',
 'Proximity Cost P1',
 'Proximity Cost P2',
 'Initial X Position P1',
 'Initial Y Position P1',
 'Initial X Position P2',
 'Initial Y Position P2',
 'Theta_a1',
 'Theta_b1',
 'Theta_c1',
 'Theta_a2',
 'Theta_b2',
 'Theta_c2',
 'Adjustment Count P2',
 'Scenario',
 'P2 Vector Cost',
 'action1_1',
 'action2_1',
 'A1_1_1_1',
 'A1_1_2_1',
 'A1_1_3_1',
 'A1_1_4_1',
 'A1_1_5_1',
 'A1_1_6_1',
 'A1_1_7_1',
 'A1_1_8_1',
 'A1_1_9_1',
 'A1_2_1_1',
 'A1_2_2_1',
 'A1_2_3_1',
 'A1_2_4_1',
 'A1_2_5_1',
 'A1_2_6_1',
 'A1_2_7_1',
 'A1_2_8_1',
 'A1_2_9_1',
 'A1_3_1_1',
 'A1_3_2_1',
 'A1_3_3_1',
 'A1_3_4_1',
 'A1_3_5_1',
 'A1_3_6_1',
 'A1_3_7_1',
 'A1_3_8_1',
 'A1_3_9_1',
 'A1_4_1_1',
 'A1_4_2_1',
 'A1_4_3_1',
 'A1_